<a href="https://colab.research.google.com/github/kk25gb/AIdea-project-file/blob/Siamese-network/Indexer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 建立索引清單

In [19]:
import json
import csv
from urllib.request import urlopen
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np
import tqdm

In [20]:
url = 'https://raw.githubusercontent.com/kk25gb/AIdea-project-file/main/stage1/vectorize_data.json'
response = urlopen(url)
data = json.loads(response.read())
df = pd.DataFrame(data).T

In [21]:
# url = 'https://raw.githubusercontent.com/kk25gb/AIdea-project-file/Siamese-network/stage2/similarity_list_stage_2.json'
# response = urlopen(url)
# similarity_list = json.loads(response.read())

In [22]:
# similarity_list

In [23]:
# url = 'https://raw.githubusercontent.com/kk25gb/AIdea-project-file/Siamese-network/stage2/column_filter.json'
# response = urlopen(url)
# column_filter = json.loads(response.read())

In [24]:
def Union_calculation(input):
  union = set()
  for values in list(input.values()):
    if len(union)==0:
      union = set(values)
    else:
      union = union & set(values)
  return union

In [40]:
similarity_list = {}
column_filter={}
for key,labels in data.items():
  row_1 = pd.DataFrame(labels).T
  mask = row_1 > 0
  filter = row_1[mask].dropna(axis=1).columns.tolist()
  column_filter[key]=filter
  filter_list = {}
  for i in filter:
    filter_list[i] = df.iloc[np.flatnonzero(df[i])].index.tolist()
  similarity_list[key] = list(set(filter_list)) #list(Union_calculation(filter_list))

In [46]:
print(list(similarity_list.keys())[0])
similarity_list[list(similarity_list.keys())[0]]

1


[369, 2, 474, 156, 559]

In [27]:
# similarity_list

In [41]:
index=3
object_list = df[column_filter[list(column_filter.keys())[index]]].loc[list(similarity_list.keys())[index]]
object_list.iloc[:].tolist()

[6, 14, 2, 3]

In [48]:
df[column_filter[list(column_filter.keys())[index]]]

,2,156,369,474,559
1,4,3,4,3,2
10,0,0,0,0,0
1000,0,0,0,0,0
1005,0,0,0,0,0
1007,0,0,0,0,0
...,...,...,...,...,...
986,0,0,0,0,0
988,0,0,0,1,0
992,0,0,0,0,0
997,0,0,0,0,0


In [108]:
# index = 100
index = list(column_filter.keys()).index('1033')
print("Object:",list(column_filter.keys())[index])
print("index:",column_filter[list(column_filter.keys())[index]])
print("values:",df[column_filter[list(column_filter.keys())[index]]].loc[list(similarity_list.keys())[index]].tolist())
print("same_index_rows:",similarity_list[list(similarity_list.keys())[index]])
index_filter = df[column_filter[list(column_filter.keys())[index]]]#.loc[similarity_list[list(similarity_list.keys())[index]]]
index_filter

Object: 1033
index: [3, 467, 557]
values: [4, 1, 5]
same_index_rows: [3, 557, 467]


,3,467,557
1,0,0,0
10,0,0,0
1000,0,0,0
1005,0,0,0
1007,0,0,0
...,...,...,...
986,0,0,0
988,0,0,0
992,0,0,0
997,0,0,0


In [109]:
pd.DataFrame(df[column_filter[list(column_filter.keys())[index]]].loc[['1073','741']])

,3,467,557
1073,2,1,8
741,2,0,8


In [110]:
filter_dict={}
for key ,value in zip(index_filter.index,index_filter.iloc[:,:].values):
    if str(value).count('0')/len(value) <= 0.5 and key != index:
        filter_dict[key] = list(value)
    # break

In [111]:
pd.DataFrame(df[column_filter[list(column_filter.keys())[index]]].loc[list(filter_dict.keys())])

,3,467,557
1025,2,2,8
1033,4,1,5
1039,4,1,5
1073,2,1,8
189,2,0,15
191,2,0,3
3,2,0,4
313,2,0,8
35,4,0,3
415,2,1,1


# 進行迭代分析

In [30]:
df[column_filter[list(column_filter.keys())[index]]].loc[list(similarity_list.keys())[index]].tolist()

[4, 4, 8, 1]

In [31]:
results = {}
for i in list(column_filter.keys()):
  index_object = column_filter[i]
  # print("Object:",i,index_object)
  compare_object = df[column_filter[i]].loc[i].tolist()
  results[i]={"key":index_object}
  compare_table = df[column_filter[i]].loc[similarity_list[i]]
  temp={}
  for key, value in zip(compare_table.iloc[:,:].index,compare_table.iloc[:,:].values):
    if key!=i:temp[key] = value.tolist()
  if len(temp)>0:
    results[i]["values"]=temp
  else:
    del results[i]
  # print('----------------------')
  # break
  

In [32]:
# compare_table.iloc[:,:].index

In [ ]:
# results

# 文章篩選

In [112]:
url = 'https://raw.githubusercontent.com/kk25gb/AIdea-project-file/Siamese-network/stage1/dataTrainComplete.json'
response = urlopen(url)
origin_assay = json.loads(response.read())

In [113]:
origin_assay['1033']

'本市斜紋夜蛾疫情升溫，請加強防治。臺中市政府、防檢局及田邊好幫手關心您。依5月份性費洛蒙監測調查，臺中市外埔區、沙鹿區、東勢區及和平區等區域斜紋夜蛾成蟲密度增加，其幼蟲食量大常造成嚴重災情，籲請農民注意，並加強防治工作，避免蟲害蔓延擴散。(1)捕殺幼蟲或摘除卵塊:田間如發現卵塊或幼蟲，應立即捕殺或移出田間。(2)全期懸掛斜紋夜蛾性費洛蒙:誘引斜紋夜蛾雄蛾，減少交尾繁殖後代，並藉以監測田間族群密度，族群密度提升應啟動防治工作。(3)田區翻耕淹水:受害嚴重之綠肥作物應盡速翻耕淹水，避免斜紋夜蛾滋生蔓延。(4)化學藥劑防治:請農友參閱防檢局農藥資訊服務網病蟲害防治(https://pesticide.baphiq.gov.tw/web/Insecticides_MenuItem5_5.aspx)，並請注意安全採收期。'

In [115]:
origin_assay['415']

'夜蛾拉警報，白粉等著來，臺南區農改場籲請加強防治。臺南區農改場、防檢局及田邊好幫手關心您。目前氣候乾燥及正值季節轉換期間，田間夜蛾類害蟲密度逐漸上升，尤以落花生田區發生嚴重。同時天氣漸轉冷涼，早晚溫差變大，白粉病發生機率提高，為確保作物正常生長及良好收成，臺南區農業改良場籲請農友注意夜蛾類害蟲及白粉病防治。常見危害夜蛾有斜紋夜蛾及甜菜夜蛾，兩者幼蟲皆晝伏夜出，雜食性，可危害作物如葉菜類、茼蒿、落花生、田菁、毛豆、豌豆、大蒜、青蔥、甘藷、玉米、蓮花及花卉等數十種作物。幼蟲食量極大，會大量啃食植株心梢及葉片，阻礙作物的正常生長，影響產量及品質，嚴重時常導致全園廢耕。白粉病是常見的作物葉部病害，感染初期通常由下位葉開始發生，葉表面出現白至淺灰之圓形小病斑，擴大癒合後即好似一層糖霜覆蓋整個葉面，影響光合作用，若遇久旱不雨的天氣，嚴重時葉片黃化，提早落葉。於春、秋兩季較乾燥及光線不足的環境下危害劇烈，由於白粉病亦會感染葉柄、嫩蔓等部位，如病害繼續進行，最終作物全株表面皆覆滿白色粉狀物而呈青白色。夜蛾類害蟲防治上應注意下列事項：(1)田間如發現夜蛾類害蟲卵塊、幼蟲，應立即捕殺或摘除(2)幼蟲及成蟲大多於太陽下山或傍晚時分開始活動，傍晚噴藥效果較佳，噴灑時應注意使用充足水量(3)參考植物保護手冊推薦用藥，選擇藥劑交互使用，以避免害蟲產生抗藥性(4)利用性費洛蒙長期誘殺雄蛾，以降低其田間密度。每公頃設5~10個點，放置高度約為離地150公分或作物生長點上方30~60公分，每月更新誘餌一次。作物種植密集地區應採共同防治，成效更佳(5)注意清園之工作，間作之作物應同時施藥，徹底清除蟲源(6)旱田作物於種植前整地翻犁後浸水一天以上，可將土中的蛹或幼蟲淹死(7)收穫前注意化學藥劑的使用，或改用蘇力菌防治，注意安全採收期，以免造成農藥殘留過量之問題(8)受害嚴重之綠肥作物或廢耕田區應儘速翻犁淹水，避免夜蛾類滋生蔓延。白粉病防治法如下：(1)保持良好通風狀況，避免莖葉過於茂密。(2)發病初期可施用葵無露或窄域油。(3)參考作物登記藥劑，由於白粉病病原菌易產生抗藥性，因此應選擇多種藥劑，輪流施用，以避免抗藥性產生，並遵守安全採收期。白粉病葉面葉背都會發生，因此防治時應注意均勻噴施藥劑於葉片兩面。藥劑防治請參考植物保護手冊登記藥劑，可於本場網站連結各項作物的相關用藥資料：http://www.t

# 匯出檔案

##儲存為JSON格式

In [34]:
import json
with open('./results.json', 'w', encoding='utf8') as f:
    json.dump(results, f, ensure_ascii=False)

## 從Colab下載檔案到電腦

In [35]:
from google.colab import files
files.download("/content/results.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>